# Collection value evolution

Libs

In [1]:
import datetime as dt
import json
import numpy as np
import pandas as pd
import requests
import sqlite3
import yaml
from requests.exceptions import HTTPError
import time

Configuration file

In [2]:
with open(r'../config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

url_discogs_api = "https://api.discogs.com"

## Retrieve the collection items

Get number of pages of collection items

In [3]:
try:
    query = {'page': 1, 'per_page': 100}
    url_request = url_discogs_api + "/users/" + config["discogs_user"] + "/collection/folders/0/releases"
    response = requests.get(url_request, params=query)
    response.raise_for_status()
    jsonResponse = response.json()
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

no_pages = jsonResponse["pagination"]["pages"] # * 50

Get the collection items

In [4]:
collection_items = []
for i in range(1, no_pages + 1):
    try:
        query = {'page': i, 'per_page': 100}
        url_request = url_discogs_api + "/users/" + config["discogs_user"] + "/collection/folders/0/releases"
        response = requests.get(url_request, params=query)
        jsonResponse = response.json()
        collection_items.append(pd.json_normalize(jsonResponse["releases"]))
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

df_collection = pd.concat(collection_items, ignore_index=True)

Subselection of columns

In [6]:
selected_columns = df_collection.columns[~df_collection.columns.isin([ "basic_information.thumb", "basic_information.cover_image",\
     "basic_information.artists", "basic_information.labels", "basic_information.formats"])]
df_collection = df_collection[selected_columns]

Write stuff to sqlite db

In [7]:
db = sqlite3.connect("test.db")
df_collection[["id", "instance_id", "date_added"]].to_sql(name="collection", con=db, if_exists='replace')
db.close()

## Retrieve marketplace values

In [18]:
query = {'curr_abbr': 'EUR'}

collection_items_value = []
for i in df_collection.index:
    url_request = url_discogs_api + "/marketplace/stats/" + str(df_collection['id'][i])
    try:
        response = requests.get(url_request, params=query)
        response.raise_for_status()

        df_item = pd.json_normalize(response.json())
        df_item['id'] = str(df_collection['id'][i])
        df_item['time_value_retrieved'] = dt.datetime.now()
        df_item = df_item.loc[:, df_item.columns != 'lowest_price']
        collection_items_value.append(df_item)

    except HTTPError as http_err:
        if response.status_code == 429:
            time.sleep(60)
    except Exception as err:
        print(f'Other error occurred: {err}')
        
df_collection_value = pd.concat(collection_items_value, ignore_index=True)

In [16]:
df_collection_value.head()

,num_for_sale,blocked_from_sale,lowest_price.value,lowest_price.currency,id,time_value_retrieved
0,3,False,65.00,EUR,9438069,2022-04-09 22:07:35.286609
1,27,False,15.00,EUR,9331164,2022-04-09 22:07:35.836057
2,5,False,4.99,EUR,6239920,2022-04-09 22:07:36.383411
3,1,False,34.99,EUR,1965832,2022-04-09 22:07:36.940612
4,12,False,27.47,EUR,403287,2022-04-09 22:07:37.536488


Add to database

In [22]:
db = sqlite3.connect("test.db")
df_collection_value.to_sql(name="collection_value", con=db, if_exists='append')
db.close()